In [1]:
import pandas as pd
import statsmodels.api as sm
import sys
sys.path.append(r'C:\Users\camde\OneDrive - Brigham Young University\IS JUNIOR CORE-Camdens-Laptop\IS 455\myfunctions')
import myfunctions as my

df_ins = pd.read_csv(r'C:\Users\camde\OneDrive - Brigham Young University\IS JUNIOR CORE-Camdens-Laptop\IS 455\univariate_automation\insurance.csv')
df_nba = pd.read_csv(r'C:\Users\camde\OneDrive - Brigham Young University\IS JUNIOR CORE-Camdens-Laptop\IS 455\nbastats\nba_salaries.csv')

In [2]:
y = df_ins['charges']
X = df_ins[['age', 'bmi', 'children']].assign(const=1)  # Add const=1 because we're in statsmodels

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     60.69
Date:                Thu, 13 Feb 2025   Prob (F-statistic):           8.80e-37
Time:                        15:11:16   Log-Likelihood:                -14392.
No. Observations:                1338   AIC:                         2.879e+04
Df Residuals:                    1334   BIC:                         2.881e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age          239.9945     22.289     10.767      0.0

In [3]:
print(f"Predicted value: ${model.predict([45, 25, 3, 1])[0]:.2f}")

Predicted value: $13814.19


###ADD IN CATEGORICAL COLUMNS###

In [4]:
df_ins = pd.read_csv(r'C:\Users\camde\OneDrive - Brigham Young University\IS JUNIOR CORE-Camdens-Laptop\IS 455\univariate_automation\insurance.csv')
df_ins = pd.get_dummies(df_ins, drop_first=True) #this drops unneeded columns when converted to bools

df_ins[df_ins.select_dtypes(bool).columns] = df_ins.select_dtypes(bool).astype(int)
df_ins.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0


In [5]:
y = df_ins['charges']
X = df_ins.drop(columns=['charges']).assign(const=1)

model = sm.OLS(y, X).fit() #ordinary least squares regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Thu, 13 Feb 2025   Prob (F-statistic):               0.00
Time:                        15:11:16   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                256.8564     11.899  

In [6]:
print(f"Prediction: ${model.predict([40, 23, 2, 1, 0, 0, 0, 1, 1])[0]:.2f}")

Prediction: $5996.80


NBA SALARY PIPELINE

In [7]:
# IMPORT DATA
df_nba = pd.read_csv(r'C:\Users\camde\OneDrive - Brigham Young University\IS JUNIOR CORE-Camdens-Laptop\IS 455\nbastats\nba_salaries.csv')

# EXPLORATORY DATA ANALYSIS: UNIVARIATE
# df_nba = my.univariate(df_nba)

#DATA PREPARATION
df_nba = my.basic_wrangling(df_nba, messages=False)
# df_nba = my.missing_fill(df_nba, 'Salary')
df_nba.dropna(inplace=True)
df_nba.drop(columns=['Team'], inplace=True)

df_nba.loc[df_nba['Position'] == 'PG-SG', 'Position'] = 'PG'
df_nba.loc[df_nba['Position'] == 'SF-SG', 'Position'] = 'SF'
df_nba.loc[df_nba['Position'] == 'SG-PG', 'Position'] = 'SG'
df_nba.loc[df_nba['Position'] == 'SF-PF', 'Position'] = 'SF'

# EXPLORATORY DATA ANALYSIS: BIVARIATE
# my.bivariate(df_nba, 'Salary')

# MODELING
y = df_nba['Salary']
X = df_nba.drop(columns=['Salary', 'Age', 'FG', 'FG%', 'FGA', '3P', '3PA']).assign(const=1)
X = pd.get_dummies(X, drop_first=True)
X[X.select_dtypes(bool).columns] = X.select_dtypes(bool).astype(int) #turns true/false to 0/1

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.575
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                     22.97
Date:                Thu, 13 Feb 2025   Prob (F-statistic):           3.72e-61
Time:                        15:11:17   Log-Likelihood:                -7445.8
No. Observations:                 433   AIC:                         1.494e+04
Df Residuals:                     408   BIC:                         1.504e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
GP          -2.089e+04   2.11e+04     -0.992      